# Data Wrangling

## Introduction <a name="introduction"></a>
- #### This notebook explores the raw data given by Census.gov and Kaggle api's as well as the data downloaded from the Colorado Department of Education. 
- #### It will guide feature extraction from the datasets to a more human readable format.
- #### Finally, it will reduce the number of datasets to a more manageable amount to aid in exploratory data analysis. 

## Table of contents
* [Introduction](#introduction)
* [1 Getting Data](#get-data)
* [2 Making Datasets](#make-datasets)
    * [2.1 Census](#make-census)
    * [2.2 Expenditures](#make-expenditures)
    * [2.3 Kaggle](#make-kaggle)
        * [2.3.1 Sub Set](#make-subset)

In [252]:
import numpy as np
import pandas as pd

# To modules necessary to load the src.data.get_raw_data
import importlib
import sys

# setting path
sys.path.append('..')
# importing
from src.data import get_raw_data, make_datasets
importlib.reload(get_raw_data);
importlib.reload(make_datasets);

## 1. Getting Data <a id="get-data"></a>
Data was obtained from the following sources
    - [SAIPE Datasets (census.gov)](https://www.census.gov/programs-surveys/saipe/data/datasets.html): The number of students whose families below the poverty level by district for the years 2010-2012 through an API.
    - [School District Revenues and Expenditures | CDE (state.co.us)](http://www.cde.state.co.us/cdefinance/revexp): Revenue and expenditures by program and district for the years 2010-2012 which is downloaded manually.
    - [Visualize the State of Public Education in Colorado | Kaggle](https://www.kaggle.com/competitions/visualize-the-state-of-education-in-colorado/data?select=2010_1YR_3YR_change.csv): Student performance, college-readiness, and demographic information by school for the years 2010-2012 through an API

In [134]:
# Load the raw data 
raw_filepath = '../data/raw/' # filepath to all raw data
get_raw_data.get_census(raw_filepath + 'census/') # loads SAIPE data sets from census.gov saved as saipe_'year'
get_raw_data.get_kaggle(raw_filepath + 'kaggle/') # obtains the Kaggle competition data with their original names

## 2 Making Datasets <a id="make-datasets"></a>
The output produced by these api's and the files downloaded are in an extremely poor format especially the expenditures files. They need to be revised and improved in order to be used for exploratory data analysis. Ultimately, we will create a module named src.make_datasets to complete this process, so that it can be scaled. We will use this notebook to investigate how to do this effectively. Furthermore, we will try to define uniform column names such as district_name, school, or county to be used later when we join them. Finally, these datasets will be saved in a tall format in the interim folder. In other words, we will concatenate each year of the same datasets to with an additional year column to distinguish them.

In [143]:
# The filepath to save usable datasets to
interim_filepath = '../data/interim/'

### 2.1 Census <a id='make-census'></a>

In [130]:
census = pd.read_csv('../data/raw/census/saipe2010.csv', index_col=0)
census.head()

,SD_NAME,SAEPOV5_17RV_PT,SAEPOV5_17V_PT,SAEPOVALL_PT,time,state,school district (unified)
0,Cheyenne County School District RE-5,39,229,1350,2010,8,1
1,Yuma School District 1,179,977,5017,2010,8,16
2,Wray School District RD-2,103,727,3958,2010,8,17
3,Idalia School District RJ-3,12,92,502,2010,8,18
4,Liberty School District J-4,7,69,507,2010,8,19


__The census data is in pretty good shape, but there are some changes that can be made__
* The census columns should be renamed to be more intuitive. We will use the following map, 
    - SD_NAME : district_name
    - SAEPOV5_17RV_PT : est_child_poverty
    - SAEPOV5_17V_PT: est_total_child
    - SAEPOVALL_PT : est_total_pop
    - time : year
* We will also drop a few columns that we don't need
    - state
    - school district (unified)

In [138]:
make_datasets.make_tall_census(raw_filepath + 'census/', interim_filepath)

In [142]:
pd.read_csv(interim_filepath + 'saipe_tall.csv', index_col=0)

,district_name,est_child_poverty,est_total_child,est_total_pop,year
0,Cheyenne County School District RE-5,39,229,1350,2010
1,Yuma School District 1,179,977,5017,2010
2,Wray School District RD-2,103,727,3958,2010
3,Idalia School District RJ-3,12,92,502,2010
4,Liberty School District J-4,7,69,507,2010
...,...,...,...,...,...
173,Wiggins School District RE-50J,80,518,2556,2012
174,Wiley School District RE-13-JT,33,192,937,2012
175,Windsor School District RE-4,362,4722,22434,2012
176,Woodland Park School District RE-2,416,3042,18577,2012


### 2.2 Expenditures <a id="make-expenditures"><a/>

In [236]:
expenditures = pd.read_csv('../data/raw/expenditures/expenditures2010.csv')
expenditures

,Unnamed: 0,Unnamed: 1,DISTRICT/,Unnamed: 3,Total,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,BOCES,Total,Support,Community,Other,Total
1,NaN,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,$,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
...,...,...,...,...,...,...,...,...
1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,NaN,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
1014,$,Amount,NaN,"4,422,714,161","3,067,303,089","42,807,954","2,194,922,404","9,727,747,609"
1015,$,Per Pupil,"798,599.5","5,538","3,841",54,"2,748","12,181"


__The expenditures datasets are incredibly messy and will need a lot of fixing up. The end result that we want to obtain is a dataframe with the following columns__
* district_name
* county
* and the amount and per pupil amount for each category of spending

Also, we will be able to drop all BOCES funding as this not applicable to our project

In [224]:
# We can drop two things
# the empty rows between each district's information
# and the Unnamed: 0 column which just explains the units for each row.
expenditures = expenditures.dropna(how='all').drop('Unnamed: 0', axis=1)
expenditures.head()

,Unnamed: 1,DISTRICT/,Unnamed: 3,Total,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,BOCES,Total,Support,Community,Other,Total
1,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
3,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
5,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"


In [225]:
# Lets rename the columns that explain things better
expenditures.columns = ['county', 'district_name', 'instruction', 'support', 'community', 'other', 'sum']
expenditures.head()

,county,district_name,instruction,support,community,other,sum
0,NaN,BOCES,Total,Support,Community,Other,Total
1,COUNTY,FPC,Instruction,Services,Services,Expenditures,Expenditures
3,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
4,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
5,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"


In [226]:
# Now we can drop the first two rows which contained column name information
expenditures = expenditures.drop([0,1])
expenditures = expenditures.reset_index(drop=True)
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,"39,962,942","23,760,636","237,704","5,474,579","69,435,862"
2,Per Pupil,"7,193.3","5,556","3,303",33,761,"9,653"
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,NaN,50.4,47.7,0.6,1.3,100.0
786,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
787,Amount,NaN,"4,422,714,161","3,067,303,089","42,807,954","2,194,922,404","9,727,747,609"
788,Per Pupil,"798,599.5","5,538","3,841",54,"2,748","12,181"


In [227]:
# All numbers have commas in them that need to be removed
expenditures = expenditures.replace(',','', regex=True)
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,39962942,23760636,237704,5474579,69435862
2,Per Pupil,7193.3,5556,3303,33,761,9653
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,NaN,50.4,47.7,0.6,1.3,100.0
786,STATE TOTALS,NaN,NaN,NaN,NaN,NaN,NaN
787,Amount,NaN,4422714161,3067303089,42807954,2194922404,9727747609
788,Per Pupil,798599.5,5538,3841,54,2748,12181


In [228]:
# The district_name column has numbers that were relevant to the BOCES funding but not our project.
# We want to be able to identify each of those and remove them.
def remove_floats(entry):
    try:
        float(entry)
        return np.nan
    except:
        return entry

In [231]:
expenditures['district_name'] = expenditures['district_name'].apply(remove_floats)
expenditures.head()

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,NaN,39962942,23760636,237704,5474579,69435862
2,Per Pupil,NaN,5556,3303,33,761,9653
3,All Funds,NaN,57.6,34.2,0.3,7.9,100.0
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN


In [117]:
# Now that they are removed, lets forward fill the district_name,
# so that we can extract the total amount for each category
# and the per pupil amount for each category
expenditures['district_name'] = expenditures['district_name'].fillna(method='ffill')
expenditures

,county,district_name,instruction,support,community,other,sum
0,ADAMS,MAPLETON 1,NaN,NaN,NaN,NaN,NaN
1,Amount,MAPLETON 1,39962942.37,23760636.02,237704.25,5474578.91,69435861.55
2,Per Pupil,MAPLETON 1,5556,3303,33,761,9653
3,All Funds,MAPLETON 1,57.6,34.2,0.3,7.9,100
4,ADAMS,ADAMS 12 FIVE STAR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
785,All Funds,UTE PASS BOCES,50.4,47.7,0.6,1.3,100
786,STATE TOTALS,UTE PASS BOCES,NaN,NaN,NaN,NaN,NaN
787,Amount,UTE PASS BOCES,4422714161.07,3067303089.38,42807954.34,2194922403.89,9727747608.68
788,Per Pupil,UTE PASS BOCES,5538,3841,54,2748,12181


In [118]:
# We noticed that there are BOCES funding entries at the bottom of the project.
# We will remove these, because they are not relevant to the project
expenditures = expenditures[~(expenditures['district_name'].str.lower().str.contains('boces'))]

In [119]:
# Now we can extract the total amounts
totals = expenditures[expenditures['county'].str.lower() == 'amount'].drop('county', axis=1).reset_index(drop=True)
totals

,district_name,instruction,support,community,other,sum
0,MAPLETON 1,39962942.37,23760636.02,237704.25,5474578.91,69435861.55
1,ADAMS 12 FIVE STAR,220263101.67,130356957.5,943009.14,83867988.86,435431057.17
2,ADAMS COUNTY 14,34792431.43,34598159.34,703485.37,7862015.31,77956091.45
3,BRIGHTON 27J,62557927.35,50921503.01,219986.8,18440305.37,132139722.53
4,BENNETT 29J,4853878.63,3540283.5,0,1004358.98,9398521.11
...,...,...,...,...,...,...
174,YUMA 1,4431710.03,3277848.7,0,1358006.84,9067565.57
175,WRAY RD-2,3532489.32,2746334.57,0,800123.51,7078947.4
176,IDALIA RJ-3,1154509.66,810850.16,0,25001.77,1990361.59
177,LIBERTY J-4,926414.66,647461.83,0,25385.95,1599262.44


In [120]:
# the per pupil amounts 
per_pupils = expenditures[expenditures['county'].str.lower() == 'per pupil'].drop('county', axis=1).reset_index(drop=True)
per_pupils

,district_name,instruction,support,community,other,sum
0,MAPLETON 1,5556,3303,33,761,9653
1,ADAMS 12 FIVE STAR,5482,3244,23,2087,10837
2,ADAMS COUNTY 14,4956,4928,100,1120,11104
3,BRIGHTON 27J,4397,3579,15,1296,9287
4,BENNETT 29J,4537,3309,0,939,8784
...,...,...,...,...,...,...
174,YUMA 1,5596,4139,0,1715,11450
175,WRAY RD-2,5378,4181,0,1218,10776
176,IDALIA RJ-3,8539,5997,0,185,14722
177,LIBERTY J-4,11068,7736,0,303,19107


In [151]:
# and the county names
counties = expenditures.loc[~(expenditures['county'].str.lower().isin(('amount', 'per pupil', 'all funds'))), ['district_name', 'county']].reset_index(drop=True)
counties.head()

,district_name,county
0,MAPLETON 1,ADAMS
1,ADAMS 12 FIVE STAR,ADAMS
2,ADAMS COUNTY 14,ADAMS
3,BRIGHTON 27J,ADAMS
4,BENNETT 29J,ADAMS


In [121]:
# Before joining these dataframes, we will first make sure that their shapes are the same
print(f'{totals.shape=}',
      f'{per_pupils.shape=}',
     f'{counties.shape=}')

totals.shape=(179, 6) per_pupils.shape=(179, 6)


In [152]:
# Now we can merge them
df = pd.merge(left=totals, right=per_pupils, on='district_name', suffixes=('_total', '_per_pupil'))
pd.merge(left=df, right=counties, on='district_name')

,district_name,instruction_total,support_total,community_total,other_total,sum_total,instruction_per_pupil,support_per_pupil,community_per_pupil,other_per_pupil,sum_per_pupil,county
0,MAPLETON 1,39962942.37,23760636.02,237704.25,5474578.91,69435861.55,5556,3303,33,761,9653,ADAMS
1,ADAMS 12 FIVE STAR,220263101.67,130356957.5,943009.14,83867988.86,435431057.17,5482,3244,23,2087,10837,ADAMS
2,ADAMS COUNTY 14,34792431.43,34598159.34,703485.37,7862015.31,77956091.45,4956,4928,100,1120,11104,ADAMS
3,BRIGHTON 27J,62557927.35,50921503.01,219986.8,18440305.37,132139722.53,4397,3579,15,1296,9287,ADAMS
4,BENNETT 29J,4853878.63,3540283.5,0,1004358.98,9398521.11,4537,3309,0,939,8784,ADAMS
...,...,...,...,...,...,...,...,...,...,...,...,...
174,YUMA 1,4431710.03,3277848.7,0,1358006.84,9067565.57,5596,4139,0,1715,11450,YUMA
175,WRAY RD-2,3532489.32,2746334.57,0,800123.51,7078947.4,5378,4181,0,1218,10776,YUMA
176,IDALIA RJ-3,1154509.66,810850.16,0,25001.77,1990361.59,8539,5997,0,185,14722,YUMA
177,LIBERTY J-4,926414.66,647461.83,0,25385.95,1599262.44,11068,7736,0,303,19107,YUMA


In [253]:
make_datasets.make_tall_expenditures(raw_filepath + 'expenditures/', interim_filepath)

In [255]:
pd.read_csv(interim_filepath + 'expenditures_tall.csv', index_col=0)

,district_name,instruction_total,support_total,community_total,other_total,sum_total,instruction_per_pupil,support_per_pupil,community_per_pupil,other_per_pupil,sum_per_pupil,county,year
0,MAPLETON 1,39962942,23760636,237704,5474579,69435862,5556.0,3303.0,33,761.0,9653.0,ADAMS,2010
1,ADAMS 12 FIVE STAR,220263102,130356958,943009,83867989,435431057,5482.0,3244.0,23,2087.0,10837.0,ADAMS,2010
2,ADAMS COUNTY 14,34792431,34598159,703485,7862015,77956091,4956.0,4928.0,100,1120.0,11104.0,ADAMS,2010
3,BRIGHTON 27J,62557927,50921503,219987,18440305,132139723,4397.0,3579.0,15,1296.0,9287.0,ADAMS,2010
4,BENNETT 29J,4853879,3540284,0,1004359,9398521,4537.0,3309.0,0,939.0,8784.0,ADAMS,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,WRAY RD-2,3933428,2857350,0,6011673,12802451,5749.0,4176.0,0,8786.0,18712.0,YUMA,2012
176,IDALIA RJ-3,1058571,957184,0,288434,2304189,6786.0,6136.0,0,1849.0,14770.0,YUMA,2012
177,LIBERTY J-4,853026,707357,0,0,1560383,10978.0,9104.0,0,0.0,20082.0,YUMA,2012
178,CHARTER SCHOOL INSTITUTE,40561158,40549832,254978,6088775,87454743,3593.0,3592.0,23,539.0,7746.0,NaN,2012
